# (each HA) V/s (metric in each hour)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import os
from pandas.io.formats.excel import ExcelFormatter
from pandas.io import gbq
from openpyxl.workbook import Workbook
from shutil import copyfile
from pathlib import Path

In [2]:
old_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_3"
old_parquet_files = glob.glob(os.path.join(old_path, "*.parquet"))
new_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_4"
new_parquet_files = glob.glob(os.path.join(new_path, "*.parquet"))

In [3]:
for file in old_parquet_files:
    df = (pd.read_parquet(file))
    filepath =  new_path + '\\'+ os.path.basename(file)   
    selected_columns = [ 'average_occupancy']  # Replace with the actual column names you want to select
    HA_df = df[selected_columns].copy()
    ungrouped_HA = pd.pivot_table(HA_df, index='date', columns='hour')
    ungrouped_HA.columns = ungrouped_HA.columns.droplevel()
    ungrouped_HA= ungrouped_HA.transpose()
    ungrouped_HA.index = pd.to_datetime(ungrouped_HA.index)
     # Assuming hour range is from 0 to 23
    ungrouped_HA.columns = list(range(24))

    # Define the new time intervals for each section
    section_intervals = [(22, 8), (8, 17), (17, 22)]
    section_dfs = []  # List to store the DataFrames for each section
    # Iterate over each section
    for section, interval in enumerate(section_intervals, start=1):
    
        if start_hour > end_hour:
            # Calculate the adjusted end column number when it exceeds 23
            adjusted_end_hour = end_hour % 24
        
            section_df = pd.concat([ungrouped_HA.iloc[:, start_hour:], ungrouped_HA.iloc[:, :adjusted_end_hour + 1]], axis=1)
            section_df.columns = ungrouped_HA.columns[start_hour:].astype(str).tolist() + ungrouped_HA.columns[:adjusted_end_hour + 1].astype(str).tolist()
        else:
            section_df = ungrouped_HA.iloc[:, start_hour:end_hour + 1].copy()
            section_df.columns = ungrouped_HA.columns[start_hour:end_hour + 1].astype(str).tolist()
    
        section_df = section_df.apply(pd.to_numeric, errors='coerce')
        section_df = section_df.transpose()
        section_df['Average'] = section_df.mean(axis=1)
        section_dfs.append(section_df)

    metric_dfs = []

    for section, section_df in enumerate(section_dfs, start=1):
        selected_columns = ['Average']  # Replace with the actual column names you want to select
        a = section_df[selected_columns].copy()
        HA_df = a[selected_columns].copy()

        quartiles = section_df.quantile([0.25, 0.5, 0.75, 1], axis=1)

        # Create a new DataFrame with quartile values
        metric_df = pd.DataFrame(quartiles.values.T, columns=['Q1', 'Q2', 'Q3', 'Q4'])
        metric_df['Average'] = section_df['Average'].values
        metric_df.index = section_df.index
        metric_dfs.append(metric_df)

    HA=ungrouped_HA.transpose()
    occupancy = pd.DataFrame(index=HA.index, columns=HA.columns)
    section_df.index=section_df.index.astype(int)
    section_dfs[0].index=section_dfs[0].index.astype(int)
    metric_dfs[0].index=metric_dfs[0].index.astype(int)
    metric_dfs[1].index=metric_dfs[1].index.astype(int)
    metric_dfs[2].index=metric_dfs[2].index.astype(int)
    for section, section_df in enumerate(metric_dfs, start=0):
        selected_quartile = 'Q2'
        # Iterate over each row index in the occupancy DataFrame
        for hour in occupancy.index:
            if hour in metric_dfs[1].index:
                q = metric_dfs[1].loc[hour, selected_quartile]
                occupancy.loc[hour] = np.where(HA.loc[hour] > q, 1, np.where(HA.loc[hour] <= q, 0, np.nan))
            if hour in metric_dfs[0].index:
                q_value = 0
                occupancy.loc[hour] = np.where(HA.loc[hour] > q_value, 1, np.where(HA.loc[hour] == q_value, 0, np.nan))
            if hour in metric_dfs[2].index:
                q = metric_dfs[2].loc[hour, selected_quartile]
                occupancy.loc[hour] = np.where(HA.loc[hour] > q, 1, np.where(HA.loc[hour] <= q, 0, np.nan))
        for col in occupancy.columns:
            # Check if the column has any value equal to 1
            if (occupancy[col] == 1).any():
            # Get the hours present in section_dfs[0] index
                hours = section_dfs[0].index
        
                # Check if the hours are present in the column index
                common_hours = hours.intersection(occupancy.index)
        
            if not common_hours.empty:
                # Update the values in the column for the common hours to 1
                occupancy.loc[common_hours, col] = 1
        occupancy["Average"]=occupancy.mean(axis=1)
        occupancy.to_parquet(filepath)
            







ValueError: Length mismatch: Expected axis has 0 elements, new values have 24 elements

In [ ]:
ungrouped_HA

In [ ]:
HA_df.describe()

In [4]:
df

SensorOcc000  SensorOcc100  SensorOcc101  SensorOcc102  \
date       hour                                                           
2022-01-01 0              NaN           NaN           NaN           NaN   
           1              NaN           NaN           NaN           NaN   
           2              NaN           NaN           NaN           NaN   
           3              NaN           NaN           NaN           NaN   
           4              NaN           NaN           NaN           NaN   
...                       ...           ...           ...           ...   
2023-06-15 19             NaN           NaN           NaN           NaN   
           20             NaN           NaN           NaN           NaN   
           21             NaN           NaN           NaN           NaN   
           22             NaN           NaN           NaN           NaN   
           23             NaN           NaN           NaN           NaN   

                 SensorOcc103  SensorOcc104  SensorOcc105  SensorOcc106  \
date       hour                                                           
2022-01-01 0              NaN           NaN           NaN           NaN   
           1              NaN           NaN           NaN           NaN   
           2              NaN           NaN           NaN           NaN   
           3              NaN           NaN           NaN           NaN   
           4              NaN           NaN           NaN           NaN   
...                       ...           ...           ...           ...   
2023-06-15 19             NaN           NaN           NaN           NaN   
           20             NaN           NaN           NaN           NaN   
           21             NaN           NaN           NaN           NaN   
           22             NaN           NaN           NaN           NaN   
           23             NaN           NaN           NaN           NaN   

                 SensorOcc107  SensorOcc108  no_of_sensors  \
date       hour                                              
2022-01-01 0              NaN           NaN              0   
           1              NaN           NaN              0   
           2              NaN           NaN              0   
           3              NaN           NaN              0   
           4              NaN           NaN              0   
...                       ...           ...            ...   
2023-06-15 19             NaN           NaN              0   
           20             NaN           NaN              0   
           21             NaN           NaN              0   
           22             NaN           NaN              0   
           23             NaN           NaN              0   

                 number_of readings  average_occupancy  
date       hour                                         
2022-01-01 0                      0                NaN  
           1                      0                NaN  
           2                      0                NaN  
           3                      0                NaN  
           4                      0                NaN  
...                             ...                ...  
2023-06-15 19                     0                NaN  
           20                     0                NaN  
           21                     0                NaN  
           22                     0                NaN  
           23                     0                NaN  

[12520 rows x 13 columns]

In [ ]:
a=pq.read_table(source="D:\\EcoBee\\Internship Final Steps\\Updated All Canada_3\\fffb105e60822fbc7b2ad502516a26a852d35623.parquet").to_pandas()